In [1]:
import numpy as np
import pandas as pd
from pprint import pprint

import gensim
from gensim.utils import simple_preprocess
from gensim import corpora, models

from nltk.stem import WordNetLemmatizer
import nltk
nltk.download('wordnet')
from nltk.tokenize import TweetTokenizer
from nltk.corpus import stopwords
stop_words = set(stopwords.words('english'))
from symspellpy.symspellpy import SymSpell, Verbosity
from sklearn.model_selection import train_test_split

import collections
import torch
import torch.nn as nn
from torch import optim
import random
import time

import warnings;
warnings.filterwarnings('ignore');

import numpy as np
import pandas as pd
from pprint import pprint

import gensim
from gensim.utils import simple_preprocess
from gensim import corpora, models

from nltk.stem import WordNetLemmatizer
import nltk
nltk.download('wordnet')
from nltk.tokenize import TweetTokenizer
from nltk.corpus import stopwords
stop_words = set(stopwords.words('english'))
from symspellpy.symspellpy import SymSpell, Verbosity
from sklearn.model_selection import train_test_split

#create spell checker/word splitter
def create_symspell(max_edit_distance, prefix_length, freq_file_path):
    # create object
    sym_spell = SymSpell(max_edit_distance, prefix_length)
    
    # create dictionary using corpus.txt
    if not sym_spell.create_dictionary(freq_file_path):
        print("Corpus file not found")
        return None
    return sym_spell

def is_valid_token(w):
    special = ['<url>','<number>', '<user>']
    return w.isalpha() or w in special

def process_tweet(tweet, tknzr, sym_spell=None, advanced=False):
    st_1 = []
    for w in tknzr.tokenize(tweet):
        #remove retweet annotation if present:
        if w == 'RT':
            if advanced:
                st_1.append('rt')
        elif w[0] == '@':
            if advanced:
                st_1.append('<user>')
        #remove hashtag symbol
        elif w[0] == '#':
            st_1.append(w[1:])
        #replace link with LINK keyword
        elif w[:4] == 'http':
            st_1.append('<url>')
        elif w.isnumeric():
            if advanced:
                st_1.append('<number>')
        else:
            st_1.append(w)
    
    st_2 = []
    
    #remove stop words and punctuation, make everything lowercase
    if sym_spell != None:
        st_2 = [sym_spell.word_segmentation(w.lower()).corrected_string 
                for w in st_1 if w.isalpha() and not w.lower() in stop_words]
    elif advanced:
        st_2 = [w.lower() for w in st_1 if is_valid_token(w) and 
                    not w.lower() in stop_words]
    else:
        st_2 = [w.lower() for w in st_1 if w.isalpha() and
                not w.lower() in stop_words]
    
    #lemmatization (converts all words to root form for standardization)
    lem = WordNetLemmatizer()
    st_3 = list(map(lambda x: lem.lemmatize(x, pos='v'), st_2))
    
    #now do word segmentation/spell check
    return ' '.join(st_3)

[nltk_data] Downloading package wordnet to /home/ashwin/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package wordnet to /home/ashwin/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [2]:
df = pd.read_csv('data/labeled_prelim.csv').dropna().astype(
        {'Relevancy':np.int32, 'Urgency':np.int32}).reset_index(drop=True)
df.pop('Id')

df2 = pd.read_csv('data/mturk_results_0-2000_processed.csv')
#df = pd.concat([df, df2], ignore_index=True)
df = df2

print(df.shape)
df.head()

(1741, 3)


,Text,Urgency,Relevancy
0,"your rescue boats, vehicles, volunteer craft ...",0,0
1,"!! According to #PBS, #Houston convention cen...",0,1
2,"""100,000 homes have been damaged by #Harvey an...",0,2
3,"""Do you need us to get a boat?"" Things I never...",0,0
4,"""Flood of epic proportions."" #HurricaneHarvey ...",0,3


In [4]:
#list of embeddings
vec_length = 50
embeddings = []

#two-way map, index->word and word->index
indexer = {}
embeddings.append(np.zeros(vec_length))
indexer[0] = 'UNK'
indexer['UNK'] = 0

embeddings.append(np.zeros(vec_length))
indexer[1] = 'PAD'
indexer['PAD'] = 1

index = 2
with open('data/glove.twitter.27B/glove.twitter.27B.%dd.txt' % vec_length) as f:
    for l in f:
        line = []
        try:
            line = l.split()
            if len(line) != vec_length+1:
                print('unmapped line: ' + str(line))
                continue
            
            word = line[0]
            embeddings.append(np.array(line[1:]).astype(np.float))
            indexer[index] = word
            indexer[word] = index
            index += 1
        except:
            print(line)
            print(index)
            break
        if index % 25000 == 0:
            print('Finished mapping %d embeddings' % index)
        
    print('Finished mapping %d embeddings' % len(embeddings))
embeddings = np.asarray(embeddings)

Finished mapping 25000 embeddings
unmapped line: ['0.45973', '-0.16703', '-1.2028', '0.41675', '0.14643', '-0.39861', '-0.35118', '-0.46944', '0.63799', '0.49569', '-0.038122', '-0.37854', '-1.2221', '-1.0439', '-1.2604', '0.01232', '-0.5159', '0.1357', '-0.093283', '0.12307', '0.48072', '-0.66419', '0.50046', '-0.58255', '0.81583', '0.72197', '-0.101', '-0.17283', '0.51572', '0.3296', '-0.0024615', '0.19475', '2.1163', '0.20636', '-1.2026', '-0.0767', '-0.1058', '-0.82518', '-0.31287', '-0.19303', '0.061489', '-0.30422', '0.75731', '-0.53688', '-0.22277', '-0.22173', '-0.37943', '0.17821', '-0.34743', '0.3064']
Finished mapping 50000 embeddings
Finished mapping 75000 embeddings
Finished mapping 100000 embeddings
Finished mapping 125000 embeddings
Finished mapping 150000 embeddings
Finished mapping 175000 embeddings
Finished mapping 200000 embeddings
Finished mapping 225000 embeddings
Finished mapping 250000 embeddings
Finished mapping 275000 embeddings
Finished mapping 300000 embeddin

In [5]:
tknzr = TweetTokenizer(strip_handles=False, reduce_len=True)
df['Text'] = df['Text'].map(lambda x: process_tweet(x, tknzr, None, True))
df['Relevancy'] = df['Relevancy'].map(lambda x: 0 if x < 1 else 1)
df['Urgency'] = df['Urgency'].map(lambda x: 0 if x < 1 else 1)
df.head()

,Text,Urgency,Relevancy
0,rescue boat vehicles volunteer craft need txwx...,0,0
1,accord pbs houston convention center need whee...,0,1
2,home damage harvey financial toll begin <url>,0,1
3,need us get boat things never think ask friend...,0,0
4,flood epic proportion hurricaneharvey inundate...,0,1


In [6]:
X = []
unk_percent = []
unk_words = set()
max_len = 0
for tweet in df['Text']:
    indices = []
    words = tweet.split()
    if len(words) > max_len:
        max_len = len(words)
    unknown = 0
    for word in words:
        if word in indexer:
            indices.append(indexer[word])
        else:
            indices.append(indexer['UNK'])
            unk_words.add(word)
            unknown += 1
        unk_percent.append(unknown/len(words))
    X.append(indices)
    

# add padding to make every tweet the same length
for i in range(len(X)):
    tweet = X[i]
    if len(tweet) < max_len:
        tweet = np.append(tweet, np.ones(max_len - len(tweet))) # b/c 'PAD' corresponds to index 1
    X[i] = tweet
    

X = np.asarray(X)
y_r = df['Relevancy'].values
y_u = df['Urgency'].values
print(np.mean(unk_percent))
# print(unk_words)
print(len(unk_words))

0.043034201184915744
459


In [7]:
# evaluates binary classification model
def calc_metrics(model, X_test, y_test):
    num_correct = 0
    num_true_pos = 0
    num_false_pos = 0
    num_false_neg = 0
    
    num_test_exs = len(X_test)

    for i in range(num_test_exs):
        y_pred = model.predict(X_test[i])
        y_gold = y_test[i]
        if y_pred == y_gold:
            num_correct += 1
            if y_gold > 0:
                num_true_pos += 1
        else:
            if y_pred == 0:
                num_false_neg += 1
            else:
                num_false_pos += 1

    accuracy = num_correct/num_test_exs
    precision = num_true_pos/(num_true_pos + num_false_pos)
    recall = num_true_pos/(num_true_pos + num_false_neg)
    f1 = 2*precision*recall/(precision+recall)

    print('accuracy: %f' % accuracy)
    print('precision: %f' % precision)
    print('recall: %f' % recall)
    print('f1: %f' % f1)

In [8]:
class RelevancyModel(nn.Module):
    def __init__(self, weights, indexer, embed_len, hidden_size, seq_len):
        super(RelevancyModel, self).__init__()
        self.indexer = indexer
        self.embed_len = embed_len
        self.hidden_size = hidden_size
        self.seq_len = seq_len
        self.embeddings = nn.Embedding.from_pretrained(weights)
        self.rnn = nn.LSTM(input_size=embed_len, 
                           hidden_size=self.hidden_size,
                           batch_first=True)
        self.fc1 = nn.Linear(self.hidden_size*self.seq_len, 2)
#         self.softmax - nn.Softmax() # only necessary if we use NLLLoss
        
    def forward(self, x, batch_size):
        x = self.embeddings(x).reshape(batch_size, self.seq_len, self.embed_len)
        x, _ = self.rnn(x) # output as (batch, seq_len, #directions*hidden)
        x = x.reshape(batch_size, self.seq_len*self.hidden_size)
        x = self.fc1(x)
        return x
    
    def predict(self, x):
        return np.argmax(self.forward(torch.from_numpy(x).long(), 1).detach().numpy())

In [9]:
class UrgencyModel(nn.Module):
    def __init__(self, weights, indexer, embed_len, hidden_size, seq_len):
        super(UrgencyModel, self).__init__()
        self.indexer = indexer
        self.embed_len = embed_len
        self.hidden_size = hidden_size
        self.seq_len = seq_len
        self.embeddings = nn.Embedding.from_pretrained(weights)
        self.rnn = nn.LSTM(input_size=embed_len, 
                           hidden_size=self.hidden_size,
                           batch_first=True)
        self.fc1 = nn.Linear(self.hidden_size*self.seq_len, 2)
#         self.softmax - nn.Softmax() # only necessary if we use NLLLoss
        
    def forward(self, x, batch_size):
        x = self.embeddings(x).reshape(batch_size, self.seq_len, self.embed_len)
        x, _ = self.rnn(x) # output as (batch, seq_len, #directions*hidden)
        x = x.reshape(batch_size, self.seq_len*self.hidden_size)
        x = self.fc1(x)
        return x
    
    def predict(self, x):
        return np.argmax(self.forward(torch.from_numpy(x).long(), 1).detach().numpy())

In [10]:
"""
X_train: 2d np array, where each row is the indices corresponding to each word of a specific tweet
y_train: 1d np array of same length as X_train with 0/1 based on relevant/not relevant or urgent/not urgent
indexer: map of word->index and index->word
embeddings: GloVe word embeddings created above
"""
def train_rnn_classifier(X_train, y_train, indexer, embeddings, rel, batch_size, epochs, lr, hidden_size, num_classes):
    try:
        start = time.time()
        embeddings = torch.from_numpy(embeddings).float()
        embed_len = len(embeddings[0])
        seq_len = len(X_train[0])
        if rel:
            rnn = RelevancyModel(embeddings, indexer, embed_len, hidden_size, seq_len)
        else:
            rnn = UrgencyModel(embeddings, indexer, embed_len, hidden_size, seq_len)
        optimizer = optim.Adam(rnn.parameters(), lr=lr)
        loss = nn.CrossEntropyLoss()
        for epoch in range(epochs):
            ex_indices = [i for i in range(len(X_train))]
            random.shuffle(ex_indices)
            total_loss = 0.0
            for idx in range(int(len(ex_indices)/batch_size)):
                cur_batch_idx = ex_indices[idx*batch_size:(idx+1)*batch_size]
                cur_X = torch.from_numpy(np.asarray([X_train[i] for i in cur_batch_idx])).long()
                cur_y = torch.from_numpy(np.asarray([y_train[i] for i in cur_batch_idx]))
                rnn.zero_grad()
                probs = rnn.forward(cur_X, batch_size)#.reshape(batch_size, 2)
    #             probs = torch.from_numpy(np.argmax(probs.detach().numpy(), axis=1))
                cur_loss = loss(probs, cur_y)
                total_loss += cur_loss
                cur_loss.backward()
                optimizer.step()
            print("Avg loss on epoch %i: %f" % (epoch+1, total_loss/len(ex_indices)))
        end = time.time()
        print("Time taken: %f seconds" % (end-start))
        return rnn
    except KeyboardInterrupt:
        end = time.time()
        print("Time taken: %f seconds" % (end-start))
        return rnn

The code below trains the relevancy and urgency classifiers and passes in the necessary hyperparameters:

- Batch size: the number of examples simultaneously evaluated
- Epochs: the number of times the model is trained on the entire training set
- Learning rate (lr): affects how much the model learns from training example (usually between .001 and .0001; anything higher will likely do very poorly)
- Hidden size: number of features in the hidden state of the RNN; basically number of features used to represent each word from the input to the RNN
- Number of classes: how many output classes there are; binary in our case because we are doing relevant/not relevant and urgent/not urgent classification


In [11]:
X_train_r, X_test_r, y_train_r, y_test_r = train_test_split(X, y_r, test_size=0.33)

print('Training Relevancy RNN Classifier')
rnn_r = train_rnn_classifier(X_train_r, y_train_r, indexer, embeddings, rel=True, 
                             batch_size=10, 
                             epochs=50, 
                             lr=.0001, 
                             hidden_size=100, 
                             num_classes=2)
print('\nRelevancy metrics:')
calc_metrics(rnn_r, X_test_r, y_test_r)

Training Relevancy RNN Classifier
Avg loss on epoch 1: 0.068230
Avg loss on epoch 2: 0.066012
Avg loss on epoch 3: 0.063008
Avg loss on epoch 4: 0.060070
Avg loss on epoch 5: 0.057887
Avg loss on epoch 6: 0.056686
Avg loss on epoch 7: 0.055477
Avg loss on epoch 8: 0.054661
Avg loss on epoch 9: 0.053667
Avg loss on epoch 10: 0.053547
Avg loss on epoch 11: 0.052828
Avg loss on epoch 12: 0.052231
Avg loss on epoch 13: 0.051536
Avg loss on epoch 14: 0.051882
Avg loss on epoch 15: 0.050419
Avg loss on epoch 16: 0.049982
Avg loss on epoch 17: 0.049605
Avg loss on epoch 18: 0.048696
Avg loss on epoch 19: 0.047956
Avg loss on epoch 20: 0.046954
Avg loss on epoch 21: 0.046595
Avg loss on epoch 22: 0.046591
Avg loss on epoch 23: 0.046334
Avg loss on epoch 24: 0.044648
Avg loss on epoch 25: 0.044244
Avg loss on epoch 26: 0.042484
Avg loss on epoch 27: 0.042403
Avg loss on epoch 28: 0.040682
Avg loss on epoch 29: 0.040298
Avg loss on epoch 30: 0.039303
Avg loss on epoch 31: 0.038309
Avg loss on ep

In [138]:
X_train_u, X_test_u, y_train_u, y_test_u = train_test_split(X, y_u, test_size=0.33)

print('Training Urgency RNN Classifier')
rnn_u = train_rnn_classifier(X_train_u, y_train_u, indexer, embeddings, rel=False, 
                             batch_size=10, 
                             epochs=20, 
                             lr=.0001, 
                             hidden_size=128, 
                             num_classes=2)

print('\nUrgency metrics:')
calc_metrics(rnn_u, X_test_u, y_test_u)

Training Urgency RNN Classifier
Avg loss on epoch 1: 0.040895
Avg loss on epoch 2: 0.029574
Avg loss on epoch 3: 0.026931
Avg loss on epoch 4: 0.026035
Avg loss on epoch 5: 0.024469
Avg loss on epoch 6: 0.023774
Avg loss on epoch 7: 0.022641
Avg loss on epoch 8: 0.021547
Avg loss on epoch 9: 0.020346
Avg loss on epoch 10: 0.018623
Avg loss on epoch 11: 0.016934
Avg loss on epoch 12: 0.015448
Avg loss on epoch 13: 0.013958
Avg loss on epoch 14: 0.012442
Avg loss on epoch 15: 0.009752
Avg loss on epoch 16: 0.009608
Avg loss on epoch 17: 0.006937
Avg loss on epoch 18: 0.005164
Avg loss on epoch 19: 0.003680
Avg loss on epoch 20: 0.003832
Time taken: 23.027568 seconds

Urgency metrics:
accuracy: 0.853659
precision: 0.393939
recall: 0.448276
f1: 0.419355
